A notebook to understand the failing tests

In [5]:
import sys, os

sys.path.append(os.path.join("..", "spooler_files"))

from spooler_fermions import *

Import the functions that are in the tests folder...

let us dig deeper into what is going wrong

In [36]:
json_dict = {
        "experiment_0": {
            "instructions": [
                ["load", [7], []],
                ["load", [2], []],
                ["measure", [2], []],
                ["measure", [6], []],
                ["measure", [7], []],
            ],
            "num_wires": 8,
            "shots": 4,
        },
    }

result_dict = {
        "backend_name": "synqs_fermionic_tweezer_simulator",
        "backend_version": "0.0.1",
        "job_id": job_id,
        "qobj_id": None,
        "success": True,
        "status": "finished",
        "header": {},
        "results": [],
    }

We can basically narrow down the problem to the load gate ...

In [47]:
def check_json_dict(json_dict):
    """
    Check if the json file has the appropiate syntax.

    Args:
        json_dict (dict): the dictonary that we will test.

    Returns:
        bool: is the expression having the appropiate syntax ?
    """
    ins_schema_dict = {
        "load": barrier_measure_schema,
        "barrier": barrier_measure_schema,
        "hop": hop_schema,
        "int": int_schema,
        "phase": int_schema,
        "measure": barrier_measure_schema,
    }
    max_exps = 50
    for e in json_dict:
        err_code = "Wrong experiment name or too many experiments"
        try:
            exp_ok = (
                e.startswith("experiment_")
                and e[11:].isdigit()
                and (int(e[11:]) <= max_exps)
            )
        except:
            exp_ok = False
            break
        if not exp_ok:
            break
        err_code, exp_ok = check_with_schema(json_dict[e], exper_schema)
        if not exp_ok:
            break
        ins_list = json_dict[e]["instructions"]
        for ins in ins_list:
            try:
                err_code, exp_ok = check_with_schema(ins, ins_schema_dict[ins[0]])
                print(ins)
                print(err_code)
                
            except Exception as e:
                err_code = "Error in instruction " + str(e)
                exp_ok = False
            if not exp_ok:
                break
        if not exp_ok:
            break
    print('Done')
    print(exp_ok)
    return err_code.replace("\n", ".."), exp_ok

In [48]:
err_msg, json_is_fine = check_json_dict(json_dict)

['load', [7], []]
[{'type': 'string', 'enum': ['load', 'measure', 'barrier']}, {'type': 'array', 'maxItems': 2, 'items': [{'type': 'number', 'minimum': 0, 'maximum': 7}]}, {'type': 'array', 'maxItems': 0}] is not of type 'object', 'boolean'

Failed validating 'type' in metaschema['allOf'][1]['properties']['items']['$dynamicRef']['allOf'][0]:
    {'$defs': {'anchorString': {'pattern': '^[A-Za-z_][-A-Za-z0-9._]*$',
                                'type': 'string'},
               'uriReferenceString': {'format': 'uri-reference',
                                      'type': 'string'},
               'uriString': {'format': 'uri', 'type': 'string'}},
     '$dynamicAnchor': 'meta',
     '$id': 'https://json-schema.org/draft/2020-12/meta/core',
     '$schema': 'https://json-schema.org/draft/2020-12/schema',
     '$vocabulary': {'https://json-schema.org/draft/2020-12/vocab/core': True},
     'properties': {'$anchor': {'$ref': '#/$defs/anchorString'},
                    '$comment': {'type': 

In [51]:
from pprint import pprint

In [52]:
pprint(barrier_measure_schema)

{'items': [{'enum': ['load', 'measure', 'barrier'], 'type': 'string'},
           {'items': [{'maximum': 7, 'minimum': 0, 'type': 'number'}],
            'maxItems': 2,
            'type': 'array'},
           {'maxItems': 0, 'type': 'array'}],
 'maxItems': 3,
 'minItems': 3,
 'type': 'array'}
